In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8042
WEBHOOK_BASE = ""
ADMIN_URL = "http://researcher-agent:8041"


# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


In [2]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def messages_handler(payload):
    connection_id = payload["connection_id"]
    asyncio.get_event_loop().create_task(agent_controller.messaging.send_message(connection_id, "This is a response from Bob"))
    print("Handle message", payload, connection_id)


message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}


def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener, message_listener], defaults=True)

In [3]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

Connection Handler Called
Connection 10bf00c7-fe28-4f15-ac2c-f598e3e60257 in State invitation
Connection ID 10bf00c7-fe28-4f15-ac2c-f598e3e60257
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '31d22c96-b531-467a-8637-867f0e9199ac', 'label': 'Health Researcher', 'serviceEndpoint': 'http://172.17.0.1:8040', 'recipientKeys': ['Hu8va1kwBxycxKa731q6FeenAmhk5s3x2X3MXXR7CckE']}
Connection Handler Called
Connection 10bf00c7-fe28-4f15-ac2c-f598e3e60257 in State request


## Copy the invitation output from 4 and to the hospital notebook you are connecting to.

In [4]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

Connection Handler Called
Connection 10bf00c7-fe28-4f15-ac2c-f598e3e60257 in State response
ACCEPT REQUEST
{'created_at': '2021-01-23 14:21:34.771903Z', 'routing_state': 'none', 'accept': 'manual', 'invitation_key': 'Hu8va1kwBxycxKa731q6FeenAmhk5s3x2X3MXXR7CckE', 'initiator': 'self', 'my_did': '8wvYweJAFRYr4LZA77hYqa', 'invitation_mode': 'once', 'updated_at': '2021-01-23 14:22:26.279182Z', 'connection_id': '10bf00c7-fe28-4f15-ac2c-f598e3e60257', 'their_label': 'Royal Infirmary of Edinburgh', 'state': 'response', 'their_did': 'ALrPXCqEuqa5wYFUDDkfeo'}
state response


In [5]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

Trust Ping {'thread_id': 'fed59ad7-f56e-4724-832d-00a9234dfa6b'}
Connection Handler Called
Connection 10bf00c7-fe28-4f15-ac2c-f598e3e60257 in State active


In [ ]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")

In [ ]:
response = await agent_controller.terminate()
print(response)